In [1]:
import os
import pandas as pd
import re
import numpy as np
import datetime

In [2]:
bilibili_list = [730603253025,730901991727,730903103301]
def bilibili_taoke_divide(sku_id,bilibili_list):
    if sku_id in bilibili_list:
        return "B站"
    else:
        return "taoke"

In [13]:
itemList = ["塞德思","灵的融","希纳露","百热朗","健眠计划","S+","琵力消","VL"]
def tag_add(itemName,itemLists):
    
        for item in itemLists:
            if item in itemName:
                return item
            else:
                continue
        return itemName

In [3]:
def get_liuliang_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    liuliang_date = re.findall(date_pattern,xls_file_name)[0]
    return liuliang_date

In [5]:
xls_list=os.listdir(r"I:\渠道数据")
liuliang_data = pd.read_csv("I:/渠道数据/"+xls_list[0])
liuliang_data["日期"] = pd.to_datetime(get_liuliang_date(xls_list[0]))

In [8]:
for _data in xls_list[1:]:
    liuliang_data_sub = pd.read_csv("I:/渠道数据/"+_data)
    liuliang_data_sub["日期"] = pd.to_datetime(get_liuliang_date(_data))
    liuliang_data = pd.concat([liuliang_data,liuliang_data_sub],axis=0,ignore_index=True)


In [25]:
from numpy import int64


liuliang_data["渠道"] = liuliang_data["商品ID"].apply(lambda x:bilibili_taoke_divide(sku_id=x,bilibili_list=bilibili_list))
liuliang_data["月份"] = liuliang_data["日期"].dt.month
liuliang_data.dropna(subset=["商品名称"],inplace=True)
liuliang_data["商品类型"] = liuliang_data["商品名称"].apply(lambda x:tag_add(itemName=x,itemLists=itemList))
liuliang_data["点击人数(即进店人数)"] = liuliang_data["点击人数(即进店人数)"].astype(int64)


In [ ]:
liuliang_data.pivot_table(values=["点击人数(即进店人数)","付款人数","付款支出费用","结算金额"],index=["月份","渠道","商品类型"],aggfunc=sum)